In [4]:
import numpy as np
import pickle

with open("txt_data", "rb") as fp:   #Pickling
    data = pickle.load(fp)
with open("txt_labels", "rb") as fp:   #Pickling
    labels = pickle.load(fp)

print(len(data), len(labels))

31702 31702


In [5]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size = 0.30, shuffle=True, random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, shuffle=True, random_state = 33)

print('split done.')

split done.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500, use_idf=True, stop_words='english')
vectorizer.fit(data)
x_train = vectorizer.transform(x_train).toarray()
x_test = vectorizer.transform(x_test).toarray()
x_valid = vectorizer.transform(x_valid).toarray()
print('> tfidVectorization done.')

> tfidVectorization done.


In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

params = {
    'kernel':['linear'],
    'C':[0.001, 0.01, 0.1, 1, 10, 100]
}
clf = GridSearchCV(SVC(), param_grid=params, cv=2, n_jobs=-1, verbose=1)
clf.fit(x_train, y_train)

clf = clf.best_estimator_
print()
print('Best params: ',clf)

y_pred = clf.predict(x_test)

print()
dt = pd.DataFrame(confusion_matrix(y_test, y_pred))
display(dt)
print(classification_report(y_test, y_pred, zero_division=0))

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 13.7min finished



Best params:  SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)



,0,1,2,3,4,5,6,7,8
0,851,15,31,30,51,11,15,40,16
1,36,1033,32,25,57,4,19,21,8
2,36,37,252,16,69,2,2,10,6
3,44,29,15,218,62,4,6,21,9
4,36,43,47,39,928,10,8,31,11
5,22,25,9,11,43,326,3,8,6
6,3,3,0,2,4,1,977,184,227
7,8,6,1,2,14,0,236,958,502
8,2,3,1,2,8,0,206,501,921


              precision    recall  f1-score   support

           0       0.82      0.80      0.81      1060
           1       0.87      0.84      0.85      1235
           2       0.65      0.59      0.62       430
           3       0.63      0.53      0.58       408
           4       0.75      0.80      0.78      1153
           5       0.91      0.72      0.80       453
           6       0.66      0.70      0.68      1401
           7       0.54      0.55      0.55      1727
           8       0.54      0.56      0.55      1644

    accuracy                           0.68      9511
   macro avg       0.71      0.68      0.69      9511
weighted avg       0.68      0.68      0.68      9511



In [8]:
import pickle

with open('CLF_svm_linear','wb') as f:
    pickle.dump(clf, f)